In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Get data and rename columns

In [24]:
request = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
my_table  = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]
my_table.rename(columns = {'Postcode':'PostalCode'},inplace=True)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [25]:
my_table.set_index('Borough').drop('Not assigned').reset_index(inplace=True)

In [26]:
my_table.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [28]:
my_table = my_table.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [29]:
my_table.loc[my_table['Neighbourhood']=='Not assigned','Neighbourhood'] = my_table.loc[my_table['Neighbourhood']=='Not assigned']['Borough'] 

Verifying the above action

In [30]:
my_table.loc[my_table['PostalCode']=='M5A']

,PostalCode,Borough,Neighbourhood
80,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [31]:
my_table.loc[my_table['Neighbourhood']=="Queen's Park"]

,PostalCode,Borough,Neighbourhood
120,M7A,Queen's Park,Queen's Park


All Done!

In [33]:
my_table.shape

(180, 3)